Steps:
1. Match images and pre-coded data
2. See if images need to be moved into new buckets
    If so, do this on the network drive
3. Load images into numpy arrays

In [241]:
import glob
import pandas as pd
import numpy as np
import os
from multiprocessing import Pool

In [242]:
path = "/Volumes/M&B/Screen_Time_Measure_Development/SNAP_IT"

In [243]:
csv_files = glob.glob(path + "/**/*.csv",recursive=True)

csv_files = [x for x in csv_files if "Backups" not in x]

In [244]:
dataframes = []
for file in csv_files:
    df = pd.read_csv(file)
    df['source'] = file
    dataframes.append(df)

combined_csv = pd.concat(dataframes)

In [245]:
combined_csv = combined_csv.loc[:,['File', 'RelativePath', 'Folder','ImageQuality','screens_n', 
                    'screen1', 'screen2', 'screen_tag','screen3', 'act1', 'act2', 
                    'act3','people', 'source']]

In [246]:
combined_csv.sort_values("source", inplace=True)

In [262]:
combined_csv.drop_duplicates(list(combined_csv.columns)[:-5],inplace=True, ignore_index=True)

In [263]:
combined_csv["screens_n"] = combined_csv.loc[:,["screen1","screen2","screen3"]].count(axis = 1)

In [279]:
combined_csv["fileok"] = False

In [280]:
volume = "/Volumes/M&B/Screen_Time_Measure_Development/SNAP_IT/"

In [281]:
def check_file(df):
    df["filepath"] = df.apply(lambda x: os.path.join(volume, str(x["Folder"]), x["RelativePath"], x["File"]), axis=1)
    df["fileok"] = df.apply(lambda x: os.path.isfile(x["filepath"]) if not x["fileok"] else x["fileok"], 
                                            axis=1)
    return df

def parallelize_df(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [282]:
%%time
combined_csv = parallelize_df(combined_csv, check_file)

CPU times: user 729 ms, sys: 394 ms, total: 1.12 s
Wall time: 8min 17s


In [283]:
combined_csv.to_csv(volume+"combineddata.csv", index=False)